In [43]:
"""
This file builds a model with limited features to restrict the amount of information the web app will need
to make a prediction
"""
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

web_app_features = ["age","race","ethnicity","sex","other_language_spoken_at_home","born_in_usa",
                "marriage_status","military_status","total_income", "employment_status",
                "insurance_status","highest_education","number_of_visits","number_of_diff","diag_amt","total_expenditure"]

In [30]:
def get_cat_num_features(df): 
    num_unique = df.nunique()
    categorical_features = num_unique[num_unique <= 10].index.tolist()
    categorical_features.remove("number_of_diff")
    numerical_features = [f for f in df.columns if f not in categorical_features]
    return categorical_features, numerical_features

def normalize_target(df, target_column):
    # medical expenditure is strongly positively skewed so best practice is to normalized it for our model
    vals = df[target_column].values 
    return np.array([0 if v == 0 else np.log(v) for v in vals])

def readin_and_split(path, target_column, features):
    df = pd.read_csv(path) 
    df = df[features]
    # remove employment status == 2
    df = df[df.employment_status != 2]
    df = df[df['age'] >= 0] 
    df = df[df['marriage_status'] >= 0]
    df = df[df['total_income'] >= 0]
    df = df[(df[["other_language_spoken_at_home","born_in_usa",
                "marriage_status","military_status","employment_status",
                "insurance_status","highest_education"]] >= -1).all(1)]
    df[target_column] = normalize_target(df, target_column)
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    return train_test_split(X, y, test_size=0.2, random_state=0)


def one_hot(df):
    cat_cols = get_cat_num_features(df)[0]
    for col in cat_cols:
        df_one_hot = pd.get_dummies(df[col], prefix = col)
        df = df.drop(columns = [col])
        df = pd.concat([df, df_one_hot], axis = 1)
    return df

In [31]:
 X_train, X_test, y_train, y_test = readin_and_split("../data/meps_data_2019_new_feats.csv", "total_expenditure",  web_app_features)

In [32]:
X_train = one_hot(X_train)
X_test = one_hot(X_test)

In [33]:
X_train.head()

,age,total_income,number_of_visits,number_of_diff,diag_amt,race_1.0,race_2.0,race_3.0,race_4.0,race_5.0,...,insurance_status_7.0,insurance_status_8.0,highest_education_1.0,highest_education_2.0,highest_education_3.0,highest_education_4.0,highest_education_5.0,highest_education_6.0,highest_education_7.0,highest_education_8.0
4063,58.0,27562.0,17.0,0.0,2.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
14188,7.0,0.0,22.0,0.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7198,13.0,0.0,20.0,0.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7648,70.0,14400.0,6.0,0.0,3.0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
26881,65.0,71346.0,12.0,0.0,3.0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [35]:
parameters = {
    "n_estimators": list(range(61, 81)),
    "max_depth": list(range(2, 8)),
    "min_samples_split": list(range(2, 8)),
    "min_samples_leaf": list(range(2, 8)),
    "random_state": [1]
}

In [36]:
xgb = GradientBoostingRegressor()
reg = GridSearchCV(xgb, parameters, scoring='neg_mean_absolute_error', n_jobs = 3, verbose=2, cv=3)

In [37]:
reg.fit(X_train, y_train)

Fitting 3 folds for each of 4320 candidates, totalling 12960 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   16.1s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  1.0min
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed:  2.3min
[Parallel(n_jobs=3)]: Done 642 tasks      | elapsed:  4.1min
[Parallel(n_jobs=3)]: Done 1007 tasks      | elapsed:  6.5min
[Parallel(n_jobs=3)]: Done 1452 tasks      | elapsed:  9.3min
[Parallel(n_jobs=3)]: Done 1979 tasks      | elapsed: 12.7min
[Parallel(n_jobs=3)]: Done 2586 tasks      | elapsed: 17.8min
[Parallel(n_jobs=3)]: Done 3275 tasks      | elapsed: 23.9min
[Parallel(n_jobs=3)]: Done 4044 tasks      | elapsed: 30.7min
[Parallel(n_jobs=3)]: Done 4895 tasks      | elapsed: 39.9min
[Parallel(n_jobs=3)]: Done 5826 tasks      | elapsed: 51.0min
[Parallel(n_jobs=3)]: Done 6839 tasks      | elapsed: 64.3min
[Parallel(n_jobs=3)]: Done 7932 tasks      | elapsed: 80.5min
[Parallel(n_jobs=3)]: Done 9107 tasks      | elapsed: 108.4mi

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=3,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7],
                         'min_samples_leaf': [2, 3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5, 6, 7],
                         'n_estimators': [61, 62, 63, 64, 65, 66, 67, 68, 69,
                                          70, 71, 72, 73, 74, 75, 76, 77, 78,
                                          79, 80],
                         'random_state': [1]},
             scoring='neg_mean_absolute_error', verbose=2)

In [38]:
reg.best_score_

-0.8093175125953037

In [40]:
pickle.dump(reg, open("MEPS_xgb_model_v2_cv", "wb"))

In [41]:
y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

In [44]:
mean_squared_error(y_train, y_pred_train)

1.3208384744997035

In [45]:
mean_absolute_error(y_train, y_pred_train)

0.7882633320638476

In [46]:
r2_score(y_train, y_pred_train)

0.8646918351689498

In [47]:
mean_squared_error(y_test, y_pred_test)

1.36800769829737

In [48]:
mean_absolute_error(y_test, y_pred_test)

0.8040725287647182

In [49]:
r2_score(y_test, y_pred_test)

0.8568363062440674